# Create map of a stream reach based on a specified NWIS site and range, including various feature layers derived from USGS NLDI navigation along the stream, its tributaries and associated drainage basins, while also extracting data for further analysis (plotting and statistics)

<h2>Quick index:</h2>
<ul>
    <li><a href=#reach>Define River Reach</a></li>
    <li><a href=#map>Go to Map</a></li>
    <li><a href=#gage_plots>USGS Gage Plots</a></li>
</ul>

## Get things set up

In [195]:
import os.path
from os import path

### Use full with of window in browser

In [196]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Enable use of Javascript

In [197]:
from IPython.display import Javascript

### Enable all variable displays within a cell
 * Terminate statement with semicolon or assign to a variable ("X =") to suppress output

In [198]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Enable numerical arrays and matrices with NumPy

In [199]:
import numpy as np

### Enable R-style DataFrames with Pandas

In [200]:
import pandas as pd

### Enable Math functions

In [201]:
import math

### Enable working with Dates and Times

In [202]:
import time
from datetime import datetime

### Enable inline MATLAB-style plotting with MatPlotLib

In [203]:
import matplotlib.pyplot as plt
%matplotlib inline

### Make plots (even matplotlib plots) prettier with Seaborn

In [204]:
import seaborn as sbn

### Enable interactive functions (especially plots) using iPyWidgets

In [205]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

### Enable geospatial DataFrames with GeoPandas (built on Fiona, which is built on GDAL/OGR)

In [206]:
import geopandas as gpd

### Enable other geospatial functions using Shapely

In [207]:
import shapely
from shapely.geometry import Point, Polygon

### Enable geometry plotting for GeoPandas dataframe using Descartes (built on Shapely and MatPlotLib)

In [208]:
import descartes

### Enable Leatlet.JS-based mapping with Folium 

In [209]:
import folium
from folium import IFrame
import folium.plugins as plugins

### Enable additional HTML features with Branca

In [210]:
import branca

### Enable HTTP requests and parsing of JSON results

In [211]:
import requests
import json

### Enable SQL database access using SQLAlchemy (and GeoAlchemy)

In [212]:
from sqlalchemy import create_engine  
from sqlalchemy import Table, Column, Integer, String, Float, DateTime, MetaData  
from sqlalchemy import select, func
from sqlalchemy.ext.declarative import declarative_base  
from sqlalchemy.event import listen
from sqlalchemy.orm import sessionmaker, relationship, backref
from geoalchemy2 import Geometry

### Define some handy geometry functions

In [213]:
def geom_distance(lat1, lon1, lat2, lon2):
    R = 6378.137 # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d # Km

In [214]:
def geom_diagonal(geom):
    lon1 = geom.total_bounds[0]
    lat1 = geom.total_bounds[1]
    lon2 = geom.total_bounds[2]
    lat2 = geom.total_bounds[3]
    d = geom_distance(lat1, lon1, lat2, lon2)
    return d # Km

In [215]:
def geom_extent(geom):
    d2 = geom_width(geom)+geom_height(geom)
    return d2 # Km

In [216]:
def geom_height(geom):
    lon1 = geom.total_bounds[0]
    lat1 = geom.total_bounds[1]
    lon2 = geom.total_bounds[2]
    lat2 = geom.total_bounds[3]
    h = geom_distance(lat1, lon1, lat2, lon1)
    return h # Km

In [217]:
def geom_width(geom):
    lon1 = geom.total_bounds[0]
    lat1 = geom.total_bounds[1]
    lon2 = geom.total_bounds[2]
    lat2 = geom.total_bounds[3]
    w = geom_distance(lat1, lon1, lat1, lon2)
    return w # Km

In [218]:
def geom_bbox(geom):
    polygon = gpd.GeoDataFrame(gpd.GeoSeries(geom.envelope), columns=['geometry'])
    return polygon

In [219]:
# In case CRS is different

def geom_bbox2(geom):
    lon_point_list = [geom.total_bounds[0],geom.total_bounds[2],geom.total_bounds[2],geom.total_bounds[0],geom.total_bounds[0]]
    lat_point_list = [geom.total_bounds[1],geom.total_bounds[1],geom.total_bounds[3],geom.total_bounds[3],geom.total_bounds[1]]
    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    crs = {'init': 'epsg:4326'}
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])    
    return polygon

In [220]:
# Try using the area of the total_bounds polygon in both degrees and meters to generate an approximate "conversion" factor

def geom_area(geom):
    factor = geom_width(geom)*geom_height(geom)/geom_bbox(geom).area
    area = factor*geom.area
    return area # Km^2

In [221]:
# Use a cartesian projection coordinate system to get true area
# *** Currently crashes kernel ***

def geom_area2(geom):
    geom_m = geom.to_crs(epsg=3857) # or 3395 (WGS 84 compliant)
    # May need to use explicit definition for 3395: 
    #   proj4.defs("EPSG:3395","+proj=merc +lon_0=0 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs")
    a = geom_m.area/10**6
    return a # Km^2

<div id='reach' />

### Define the river reach by setting 
* Starting location (NWIS Station)
* Up/down-stream distance ranges (Km)
* Date range (for properties)

In [307]:
def run_all_below(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
    
run_button = widgets.Button(description=" Rerun all cells below ", button_style='warning') 
run_button.on_click(run_all_below)

start_date = datetime(2019, 1, 1) # <== Earliest start date here
end_date = datetime.now()
dates = pd.date_range(start_date, end_date, freq='D')
date_options = [(date.strftime(' %m/%d/%y '), date) for date in dates]
date_index = (0, len(date_options)-1)

date_range_slider = widgets.SelectionRangeSlider(
    options=date_options,
    index=date_index,
    description = "Date range: ",
    orientation = 'horizontal',
    layout = {'width': '500px'}
)

reach = widgets.VBox(
    [
        widgets.Label(
            value='Select NWIS Station and range (Km):\n'
        ),
        widgets.Dropdown(
            options=[
                ('Huntington','USGS-03206000'), # <== Add other NWIS stations here
                ('Charleston','USGS-03198000')
            ], 
            value='USGS-03206000', # <== Default goes here
            description='NWIS Site:'
        ),
        widgets.IntSlider(
            value=25,
            min=0,
            max=100,
            step=5,
            description='Upstream:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        ),
        widgets.IntSlider(
            value=25,
            min=0,
            max=100,
            step=5,
            description='Downstream:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        ),
        date_range_slider,
        run_button
    ]
)
reach.layout.border = 'solid'
reach.layout.width ='50%'
display(reach)

<IPython.core.display.Javascript object>

In [308]:
NWIS_SITE = reach.children[1].value
UM_DIST = reach.children[2].value
DM_DIST = reach.children[3].value
BEGIN_DATE = reach.children[4].value[0]
END_DATE = reach.children[4].value[1]

In [309]:
print("\nReach ==> NWIS Station: {0} + {1} Km upstream and {2} Km downstream".format(NWIS_SITE, UM_DIST, DM_DIST))
print("\nBegin Date: {0}, End Date: {1}\n".format(BEGIN_DATE,END_DATE))


Reach ==> NWIS Station: USGS-03206000 + 25 Km upstream and 25 Km downstream

Begin Date: 2019-01-01 00:00:00, End Date: 2020-05-20 00:00:00



### URLs for REST Web Services

In [310]:
USGS_NLDI_WS = "https://labs.waterdata.usgs.gov/api/nldi/linked-data" # USGS NLDI REST web services
NWIS_SITE_URL = USGS_NLDI_WS+"/nwissite/"+NWIS_SITE 
NWIS_SITE_NAV = NWIS_SITE_URL+"/navigate"
TNM_WS = "https://hydro.nationalmap.gov/arcgis/rest/services" # The National Map REST web services
ARCGIS_WS = "http://server.arcgisonline.com/arcgis/rest/services" # ARCGIS Online REST web services

### Set Input (optional) and Output Directories
  * TODO: Need option to upload files to Input Directory

In [311]:
IN_DIR = "data/"+NWIS_SITE
OUT_DIR = IN_DIR+"/out"

### Create output directory if it does not already exist

In [312]:
%mkdir -p {OUT_DIR}

### Get Lat/Lon coordinates of starting site (NWIS station)

In [313]:
nwis_site_json = gpd.read_file(NWIS_SITE_URL)
nwis_site_geom = nwis_site_json.iloc[0]['geometry']
nwis_site_coord = [nwis_site_geom.y, nwis_site_geom.x]

### Set up a local file-based SQLite/Spatialite database
* TODO: 
  * Only delete DB file if confirmed (e.g., known to be incomplete or invalid for some reason)

In [314]:
spatialite_db_filename = "spatialite_u{0:03d}d{1:03d}_{2:4d}{3:02d}{4:02d}_{5:4d}{6:02d}{7:02d}.db".format(UM_DIST,DM_DIST,BEGIN_DATE.year,BEGIN_DATE.month,BEGIN_DATE.day,END_DATE.year,END_DATE.month,END_DATE.day)
if os.path.isfile(OUT_DIR+'/'+spatialite_db_filename):
    #os.remove(OUT_DIR+'/'+spatialite_db_filename)
    DB_NOT_FOUND = False
else:
    DB_NOT_FOUND = True
spatialite_engine = create_engine('sqlite:///{0}/{1}'.format(OUT_DIR,spatialite_db_filename))

In [315]:
def load_spatialite(dbapi_conn, connection_record):
    dbapi_conn.enable_load_extension(True)
    dbapi_conn.load_extension('/opt/conda/lib/mod_spatialite.so')
    
listen(spatialite_engine, 'connect', load_spatialite)

In [316]:
spatialite_engine._metadata = MetaData(bind=spatialite_engine)
spatialite_engine._metadata.reflect(spatialite_engine)

In [317]:
spatialite_conn = spatialite_engine.connect()

In [318]:
spatialite_conn.execute(select([func.InitSpatialMetaData()]));

In [319]:
Base = declarative_base()

class Site(Base):
    __tablename__ = 'site'
    id = Column(Integer, primary_key=True)
    type = Column(String)
    name = Column(String)
    desc = Column(String)
    url = Column(String)
    comid = Column(String)
    geom = Column(Geometry(geometry_type='POINT',management=True))

if (DB_NOT_FOUND):
    Site.__table__.create(spatialite_engine, checkfirst=True)

In [320]:
class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    site_id = Column(String)
    type = Column(String)
    datetime = Column(DateTime)
    property = Column(String)
    value = Column(Float)
    units = Column(String)

if (DB_NOT_FOUND):
    Measurement.__table__.create(spatialite_engine, checkfirst=True)

In [321]:
Spatialite_Session = sessionmaker(bind=spatialite_engine)
spatialite_session = Spatialite_Session()

if (DB_NOT_FOUND):
    anchor_site = Site(type='ANCHOR',name=NWIS_SITE,desc='Anchor site for reach',url=NWIS_SITE_URL,geom='POINT({0:.4f} {1:.4f})'.format(nwis_site_geom.x,nwis_site_geom.y))
    spatialite_session.add(anchor_site)
    spatialite_session.commit()

#spatialite_session.rollback()
test_measurement = Measurement(site_id="TEST SITE",type="TEST",datetime=datetime.strptime("2020-05-01 07:57:00", "%Y-%m-%d %H:%M:%S"),property="Temperature",value=87.5)
spatialite_session.add(test_measurement)
spatialite_session.commit()
pd.read_sql_query("select site_id as Site, datetime as Datetime, property as Property, value as Value from measurement where type='TEST'", spatialite_engine)

## Generate map

In [322]:
river_map = folium.Map(nwis_site_coord,zoom_start=10,tiles=None)
plugins.ScrollZoomToggler().add_to(river_map);
plugins.Fullscreen(
    position='bottomright',
    title='Full Screen',
    title_cancel='Exit Full Screen',
    force_separate_button=True
).add_to(river_map);

### Add NWIS and WQP sites within reach using NLDI web services at USGS

In [323]:
# Popup parameters

width = 500
height = 120
max_width = 1000

# Main Stream

folium.GeoJson(NWIS_SITE_NAV+"/UM?distance="+str(UM_DIST),name="Main Stream (up)",show=True,control=False).add_to(river_map);
folium.GeoJson(NWIS_SITE_NAV+"/DM?distance="+str(DM_DIST),name="Main Stream (down)",show=True,control=False).add_to(river_map);

# NWIS Sites

fg_nwis = folium.FeatureGroup(name="USGS (NWIS) Sites",overlay=True,show=False)
color = 'darkred'
icon = 'dashboard'
        
nwis_sites_dm = gpd.read_file(NWIS_SITE_NAV+"/DM/nwissite?distance="+str(DM_DIST))
nwis_sites_um = gpd.read_file(NWIS_SITE_NAV+"/UM/nwissite?distance="+str(UM_DIST))
nwis_sites = gpd.GeoDataFrame(pd.concat([nwis_sites_dm,nwis_sites_um], ignore_index=True), crs=nwis_sites_dm.crs) # TODO: eliminate duplicate for anchor site

for i, nwis_site in nwis_sites.iterrows():
    coord = [nwis_site.geometry.y,nwis_site.geometry.x]
    label = 'NWIS Station: '+nwis_site.identifier+" ("+str(i)+")"
    html = label
    html += '<br>{0:s}'.format(nwis_site['name'])
    html += '<br><a href=\"{0:s}\",target=\"_blank\">{1:s}</a>'.format(nwis_site.uri,nwis_site.uri)
    html += '<br>Lat: {0:.4f}, Lon: {1:.4f}'.format(nwis_site.geometry.y,nwis_site.geometry.x)
    html += '<br>Comid: {0:s}'.format(nwis_site.comid)
    iframe = folium.IFrame(html,width=width,height=height)
    popup = folium.Popup(iframe,max_width=max_width)
    fg_nwis.add_child(folium.Marker(location=coord,icon=folium.Icon(color=color,icon=icon),popup=popup,tooltip=label));
    
    if (DB_NOT_FOUND):
        nwis_site = Site(type='NWIS',name=nwis_site.identifier,desc=nwis_site['name'],url=nwis_site.uri,comid=nwis_site.comid, \
                         geom='POINT({0:.4f} {1:.4f})'.format(nwis_site.geometry.x,nwis_site.geometry.y))
        spatialite_session.add(nwis_site)
    
if (DB_NOT_FOUND): spatialite_session.commit()
        
fg_nwis.add_to(river_map)

# WQP Stations
        
fg_wqp = folium.FeatureGroup(name="WQP Stations",overlay=True,show=False)
color = 'darkgreen'
radius = 3
        
wqp_sites_dm = gpd.read_file(NWIS_SITE_NAV+"/DM/wqp?distance="+str(DM_DIST))
wqp_sites_um = gpd.read_file(NWIS_SITE_NAV+"/UM/wqp?distance="+str(UM_DIST))
wqp_sites = gpd.GeoDataFrame(pd.concat([wqp_sites_dm,wqp_sites_um], ignore_index=True), crs=wqp_sites_dm.crs)

for i, wqp_site in wqp_sites.iterrows():
    coord = [wqp_site.geometry.y,wqp_site.geometry.x]
    label = 'WQP Station: '+wqp_site.identifier+" ("+str(i)+")"
    html = label
    html += '<br>{0:s}'.format(wqp_site['name'])
    html += '<br><a href=\"{0:s}\",target=\"_blank\">{1:s}</a>'.format(wqp_site.uri,wqp_site.uri)
    html += '<br>Lat: {0:.4f}, Lon: {1:.4f}'.format(wqp_site.geometry.y,wqp_site.geometry.x)
    html += '<br>Comid: {0:s}'.format(wqp_site.comid)
    iframe = folium.IFrame(html,width=width,height=height)
    popup = folium.Popup(iframe,max_width=max_width)
    fg_wqp.add_child(folium.CircleMarker(location=coord,radius=radius,color=color,popup=popup,tooltip=label));
    
    if(DB_NOT_FOUND):
        wqp_site = Site(type='WQP',name=wqp_site.identifier,desc=wqp_site['name'],url=wqp_site.uri,comid=wqp_site.comid, \
                        geom='POINT({0:.4f} {1:.4f})'.format(wqp_site.geometry.x,wqp_site.geometry.y))
        spatialite_session.add(wqp_site)

if (DB_NOT_FOUND): spatialite_session.commit()

fg_wqp.add_to(river_map);

### Add HUC12 Pour Points, *differential* drainage basins, HUC12 and HUC10 boundaries associated with each

In [324]:
fg_huc12pp = folium.FeatureGroup(name="HUC12 Pour Points",overlay=True,show=False)
fg_basins = folium.FeatureGroup(name="Drainage Basins",overlay=True,show=False)
#fg_wbd8 = folium.FeatureGroup(name="HUC8 Boundaries",overlay=True,show=False)
fg_wbd10 = folium.FeatureGroup(name="HUC10 Boundaries",overlay=True,show=False)
fg_wbd12 = folium.FeatureGroup(name="HUC12 Boundaries",overlay=True,show=False)

#huc8_list = []
huc10_list = []
huc12_list = []

color = 'darkblue'
radius = 3
        
try:
    huc12pp_sites_dm = gpd.read_file(NWIS_SITE_NAV+"/DM/huc12pp?distance="+str(DM_DIST),driver='GeoJSON')
except Exception as ex:
    print("An exception of type {0} occurred. Arguments:\n{1!r}".format(type(ex).__name__, ex.args))
    huc12pp_sites_dm = gpd.GeoDataFrame()
    
try:
    huc12pp_sites_um = gpd.read_file(NWIS_SITE_NAV+"/UM/huc12pp?distance="+str(UM_DIST),driver='GeoJSON')
except Exception as ex:
    print("An exception of type {0} occurred. Arguments:\n{1!r}".format(type(ex).__name__, ex.args))
    huc12pp_sites_um = gpd.GeoDataFrame()
    
huc12pp_sites = gpd.GeoDataFrame(pd.concat([huc12pp_sites_dm,huc12pp_sites_um], ignore_index=True), crs=huc12pp_sites_dm.crs)

n_segs = len(huc12pp_sites)-1

# Sort sites by decreasing area of drainage basin
# TODO: May want to store drainage basins in a list (and/or in the database)
def get_area(x):
    x_basin = gpd.read_file(USGS_NLDI_WS+"/comid/"+x+"/basin")
    return int(round(x_basin.iloc[0].geometry.area,3)*1000)  

huc12pp_sites['area']=huc12pp_sites.apply(lambda x: get_area(x.comid), axis=1)
huc12pp_sites.set_index(['area'],inplace=True,drop=True)
huc12pp_sites.sort_index(inplace=True,ascending=False)

i = 0

for area, huc12pp_site in huc12pp_sites.iterrows():
    
    # Add to HUC12 PP to Site table in database
    if (DB_NOT_FOUND):
        huc12pp = Site(type='HUC12PP',name=huc12pp_site.identifier,desc=huc12pp_site['name'],url=huc12pp_site.uri,comid=huc12pp_site.comid, \
                       geom='POINT({0:.4f} {1:.4f})'.format(huc12pp_site.geometry.x,huc12pp_site.geometry.y))
        spatialite_session.add(huc12pp)

    # Get HUC12 PP drainage basin
    basin_url = USGS_NLDI_WS+"/comid/{0:s}/basin".format(huc12pp_site.comid)
    
    try:
        basin = gpd.read_file(basin_url,driver='GeoJSON')
    except Exception as ex:
        print("An exception of type {0} occurred. Arguments:\n{1!r}".format(type(ex).__name__, ex.args))
        i = i + 1
        continue

    basin_area = geom_area(basin)
    basin_diff_area = basin_area
    
    # Get HUC12 watershed boudary (WBD)
    wbd12_url = TNM_WS+"/wbd/MapServer/6/query?where=HUC12%3D%27{0:s}%27&outFields=NAME%2CHUC12%2CSHAPE_Length&f=geojson".format(huc12pp_site.identifier)
    
    try:
        wbd12 = gpd.read_file(wbd12_url,driver='GeoJSON')
    except Exception as ex:
        print("An exception of type {0} occurred. Arguments:\n{1!r}".format(type(ex).__name__, ex.args))
        i = i + 1
        continue

    if i < n_segs:
        # Add HUC12 boundary a feature layer
        style_function = lambda x: {'color': 'darkgreen', 'weight': 1, 'fillColor': 'green', 'fillOpacity': 0.1}
        highlight_function = lambda x: {'color':'yellow', 'weight':2}
        tooltip = "HUC12: {0:s} ({1:s}), Area: {2:.2f}".format(wbd12.iloc[0].HUC12,wbd12.iloc[0].NAME,geom_area(wbd12)[0])
        wbd12_feature = folium.GeoJson(wbd12.iloc[0].geometry,style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
        fg_wbd12.add_child(wbd12_feature);        
        huc12_list.append(huc12pp_site.identifier)

    # Get HUC10 containing HUC12 and add that to another feature layer 
    huc10_identifier = huc12pp_site.identifier[:-2]

    if (huc10_identifier not in huc10_list):
        wbd10_url = TNM_WS+"/wbd/MapServer/5/query?where=HUC10%3D%27{0:s}%27&outFields=NAME%2CHUC10%2CSHAPE_Length&f=geojson".format(huc10_identifier)
        try:
            wbd10 = gpd.read_file(wbd10_url)
        except:
            pass
        else:
            basin_huc10_overlap = gpd.overlay(wbd10,basin,how='intersection')  

            if i < n_segs:
                style_function = lambda x: {'color': 'darkgreen', 'weight': 1, 'fillColor': 'green', 'fillOpacity': 0.05}
                highlight_function = lambda x: {'color':'yellow', 'weight':2}
                tooltip = "HUC10: {0:s} ({1:s}), Area: {2:.2f}".format(wbd10.iloc[0].HUC10,wbd10.iloc[0].NAME,geom_area(wbd10)[0])
                wbd10_feature = folium.GeoJson(basin_huc10_overlap.iloc[0].geometry,style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
                fg_wbd10.add_child(wbd10_feature);                 
                huc10_list.append(huc10_identifier)

#     # Get HUC8 containing HUC12 and add that to another feature layer 
#     huc8_identifier = huc12pp_site.identifier[:-4]

#     if (huc8_identifier not in huc8_list):
#         wbd8_url = TNM_WS+"/wbd/MapServer/4/query?where=HUC8%3D%27{0:s}%27&outFields=NAME%2CHUC8%2CSHAPE_Length&f=geojson".format(huc8_identifier)
#         try:
#             wbd8 = gpd.read_file(wbd8_url)
#         except:
#             pass
#         else:
#             basin_huc8_overlap = gpd.overlay(wbd8,basin,how='intersection')  

#             if i < n_segs:
#                 style_function = lambda x: {'color': 'darkgreen', 'weight': 1, 'fillColor': 'green', 'fillOpacity': 0.05}
#                 highlight_function = lambda x: {'color':'yellow', 'weight':2}
#                 tooltip = "HUC8: {0:s} ({1:s}), Area: {2:.2f}".format(wbd8.iloc[0].HUC8,wbd8.iloc[0].NAME,geom_area(wbd8)[0])
#                 wbd8_feature = folium.GeoJson(basin_huc8_overlap.iloc[0].geometry,style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
#                 fg_wbd8.add_child(wbd8_feature);                 
#                 huc8_list.append(huc8_identifier)

    if i > 0:
        # Generate and show difference (polygon) between sussessive drainage basins
        basin_diff = gpd.overlay(basin_prev,basin,how='difference')  
        basin_diff_area = geom_area(basin_diff)
        style_function = lambda x: {'color': 'red', 'weight': 1, 'fillColor': 'blue', 'fillOpacity': 0.1}
        highlight_function = lambda x: {'color':'yellow', 'weight':3}
        tooltip = "Differential Drainage Basin for HUC12 Pour Point: {0:s} ({1:s}), Area: {2:.2f}".format(wbd12.iloc[0].HUC12,wbd12.iloc[0].NAME,basin_diff_area[0])
        basin_diff_feature = folium.GeoJson(basin_diff.iloc[0].geometry.buffer(-0.001).buffer(0.001),style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
        fg_basins.add_child(basin_diff_feature);
        
        if i == n_segs:
            # Show large basin of first (highest upstream) pour point
            style_function = lambda x: {'color': 'gray', 'weight': 1, 'fillColor': 'gray', 'fillOpacity': 0.1}
            highlight_function = lambda x: {'color':'yellow', 'weight':1}
            tooltip = "Total Drainage Basin for HUC12 Pour Point: {0:s} ({1:s}), Area: {2:.2f}".format(wbd12.iloc[0].HUC12,wbd12.iloc[0].NAME,basin_area[0])
            basin_feature = folium.GeoJson(basin.iloc[0].geometry,style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
            fg_basins.add_child(basin_feature);
            
    basin_prev = basin

    # HUC12 Pour Point markers
    coord = [huc12pp_site.geometry.y,huc12pp_site.geometry.x]
    label = 'Pour Point for HUC12: '+wbd12.iloc[0].NAME
    html = label
    html += '<br>Indentifier: {0:s}'.format(huc12pp_site.identifier)
    html += '<br>Lat: {0:.2f}, Lon: {1:.2f}'.format(huc12pp_site.geometry.y,huc12pp_site.geometry.x)
    html += '<br>Comid: {0:s}'.format(huc12pp_site.comid)
    html += '<br>Area Total: {0:.2f}'.format(basin_area[0])
    html += '<br>Area Difference: {0:.2f}'.format(basin_diff_area[0])
    iframe = folium.IFrame(html,width=width,height=height)
    popup = folium.Popup(iframe,max_width=max_width)
    fg_huc12pp.add_child(folium.CircleMarker(location=coord,radius=radius,color=color,popup=popup,tooltip=label));
    
    i = i + 1
    
if (DB_NOT_FOUND): spatialite_session.commit()

fg_huc12pp.add_to(river_map);
fg_basins.add_to(river_map);
#fg_wbd8.add_to(river_map);
fg_wbd10.add_to(river_map);
fg_wbd12.add_to(river_map);

An exception of type HTTPError occurred. Arguments:
()
An exception of type HTTPError occurred. Arguments:
()
An exception of type HTTPError occurred. Arguments:
()


### Identify other HUC10s that are contained in the same HUC8 as the HUC12 Pour Point and the associated drainage basin

In [325]:
# i = 0
# n_segs = len(huc12pp_sites)

# for area, huc12pp_site in huc12pp_sites.iterrows():
#     if i >= n_segs - 1:
#         break
        
#     basin_url = USGS_NLDI_WS+"/comid/{0:s}/basin".format(huc12pp_site.comid)    
#     try:
#         basin = gpd.read_file(basin_url,driver='GeoJSON')
#     except:
#         i = i + 1
#         continue

#     # Get HUC10 watershed boundaries sharing the same HUC8
#     huc10_plus_url = TNM_WS+"/wbd/MapServer/5/query?where=HUC10%20LIKE%20%27{0:s}%25%27&outFields=NAME%2CHUC10%2CSHAPE_Length&f=geojson".format(huc12pp_site.identifier[:-4])

#     try:
#         huc10_plus = gpd.read_file(huc10_plus_url,driver='GeoJSON')
#     except:
#         i = i + 1
#         continue
        
#     huc10_basin_overlap = gpd.overlay(huc10_plus,basin,how='intersection')
    
#     if (not huc10_basin_overlap.empty):
#         for k, huc10_in_basin in huc10_basin_overlap.iterrows():
#             huc10_wbd_url = TNM_WS+"/wbd/MapServer/5/query?where=HUC10%3D%27{0:s}%27&outFields=NAME%2CHUC10%2CSHAPE_Length&f=geojson".format(huc10_in_basin.HUC10)
#             huc10_wbd = gpd.read_file(huc10_wbd_url,driver='GeoJSON')
#             huc10_overlap = gpd.overlay(huc10_wbd,basin,how='intersection')
#             if ((not huc10_overlap.empty) and (huc10_overlap.iloc[0].geometry.area > 0.001) and (huc10_in_basin.HUC10 not in huc10_list)):
#                 huc10_list.append(huc10_in_basin.HUC10)                        
#     i = i + 1

### Add HUC12s that are contained in the same HUC10 as the HUC12 Pour Point and its associated drainage basin

In [326]:
fg_huc12_plus = folium.FeatureGroup(name="Other HUC12s in HUC10",overlay=True,show=False)
        
i = 0
n_segs = len(huc12pp_sites)

for area, huc12pp_site in huc12pp_sites.iterrows():
    if i >= n_segs - 1:
        break
        
    basin_url = USGS_NLDI_WS+"/comid/{0:s}/basin".format(huc12pp_site.comid)    
    try:
        basin = gpd.read_file(basin_url,driver='GeoJSON')
    except:
        i = i + 1
        continue

    # Get HUC12 watershed boundaries sharing the same HUC10
    huc12_plus_url = TNM_WS+"/wbd/MapServer/6/query?where=HUC12%20LIKE%20%27{0:s}%25%27&outFields=NAME%2CHUC12%2CSHAPE_Length&f=geojson".format(huc12pp_site.identifier[:-2])

    try:
        huc12_plus = gpd.read_file(huc12_plus_url,driver='GeoJSON')
    except:
        i = i + 1
        continue
        
    huc12_basin_overlap = gpd.overlay(huc12_plus,basin,how='intersection')
    
    if (not huc12_basin_overlap.empty):
        for k, huc12_in_basin in huc12_basin_overlap.iterrows():
            huc12_wbd_url = TNM_WS+"/wbd/MapServer/6/query?where=HUC12%3D%27{0:s}%27&outFields=NAME%2CHUC12%2CSHAPE_Length&f=geojson".format(huc12_in_basin.HUC12)
            huc12_wbd = gpd.read_file(huc12_wbd_url,driver='GeoJSON')
            huc12_overlap = gpd.overlay(huc12_wbd,basin,how='intersection')
            if ((not huc12_overlap.empty) and (huc12_overlap.iloc[0].geometry.area > 0.001) and (huc12_in_basin.HUC12 not in huc12_list)):
                huc12_list.append(huc12_in_basin.HUC12)
                # Add HUC12 WBD boundary
                style_function = lambda x: {'color': 'darkgreen', 'weight': 1, 'fillColor': 'green', 'fillOpacity': 0.05}
                highlight_function = lambda x: {'color':'yellow', 'weight':2}
                tooltip = "HUC12: {0:s} ({1:s}), Area: {2:.2f}".format(huc12_wbd.iloc[0].HUC12,huc12_wbd.iloc[0].NAME,geom_area(huc12_wbd)[0])
                huc12_plus_feature = folium.GeoJson(huc12_wbd.iloc[0].geometry,style_function=style_function,highlight_function=highlight_function,tooltip=tooltip)
                fg_huc12_plus.add_child(huc12_plus_feature);        
                        
    i = i + 1

fg_huc12_plus.add_to(river_map);

### Add HUC12s that are contained in nearby HUC10s (in same HUC8 as the HUC12 Pour Point) and in the associated drainage basin
* TODO: How to exclude more distant HUC10s?

### Add tributaries upstream of HUC12 Pour Points

In [327]:
fg_utpp = folium.FeatureGroup(name="Tribs upstream of PPs",overlay=True,show=False)

for huc12 in huc12_list:
    wbd12_url = TNM_WS+"/wbd/MapServer/6/query?where=HUC12%3D%27{0:s}%27&f=geojson".format(huc12)
    
    try:
        wbd12 = gpd.read_file(wbd12_url)
    except:
        continue
        
    distance = int(round(geom_diagonal(wbd12),0)) # May need to exend for winding streams
    #distance = 35
    tribs = folium.GeoJson(USGS_NLDI_WS+"/huc12pp/{0:s}/navigate/UT?distance={1:d}".format(huc12,distance))
    fg_utpp.add_child(tribs);
    
fg_utpp.add_to(river_map);

### Add water quality (WQP) properties

In [328]:
wqp_begin_date = "{0:02d}-{1:02d}-{2:4d}".format(BEGIN_DATE.month,BEGIN_DATE.day,BEGIN_DATE.year)
wqp_end_date = "{0:02d}-{1:02d}-{2:4d}".format(END_DATE.month,END_DATE.day,END_DATE.year)

In [329]:
if (DB_NOT_FOUND):
    wqp_sites = spatialite_session.query(Site).filter_by(type='WQP')

    for wqp_site in wqp_sites:
        wqp_properties = pd.read_csv("https://www.waterqualitydata.us/data/Result/search?siteid="+wqp_site.name+"&startDateLo="+wqp_begin_date+"&startDateHi="+wqp_end_date+"&mimeType=csv")

        for i, wqp_property in wqp_properties.iterrows():
            try:
                value = float(wqp_property['ResultMeasureValue'])
                measurement = Measurement(site_id=wqp_site.name,type='DISCRETE',datetime=datetime.strptime(wqp_property['ActivityEndDate']+' '+wqp_property['ActivityEndDate/Time'],"%Y-%m-%d %H:%M:%S"),property=wqp_property['CharacteristicName'],value=value)
                spatialite_session.add(measurement)
            except:
                continue
        
    spatialite_session.commit()

In [330]:
property_table = pd.read_sql_query('select site_id as Site, ST_X(geom) as Lon, ST_Y(geom) as Lat, property as Property, value as Value from measurement join site on site.name = site_id', spatialite_engine)
property_table = property_table[property_table['Value'].notna()]

In [331]:
summary_table = pd.read_sql_query('select site_id as Site, ST_X(geom) as Lon, ST_Y(geom) as Lat, property as Property, count(value) as Count, min(value) as Min, max(value) as Max, avg(value) as Avg from measurement join site on site.name = site_id group by site_id, property', spatialite_engine)
summary_table = summary_table[summary_table['Avg'].notna()]

In [332]:
property_counts = property_table.groupby('Property').count()
#n_sites = wqp_sites.count()['source']
top_properties = property_counts[property_counts["Site"]>=40].sort_values('Site',ascending=False).index 
#top_properties

In [333]:
def val2color(min,value,max):
    fraction = int((value-min)/(max-min)*255)
    return '#{0:02X}00{1:02X}'.format(fraction,255-fraction)

fg_wqp_properties = folium.FeatureGroup(name='WQP Properties',overlay=True,show=True)

for property in top_properties:
    
    mean_val = property_table.query("Property == '"+property+"'")['Value'].mean()
    min_val = property_table.query("Property == '"+property+"'")['Value'].min()
    max_val = property_table.query("Property == '"+property+"'")['Value'].max()
    std_val = property_table.query("Property == '"+property+"'")['Value'].std()

    try:
        if (math.isnan(mean_val) or math.isnan(std_val) or math.isnan(min_val) or math.isnan(max_val)):
            continue
    except:
        continue

    width = 500
    height = 90
    max_width = 1000 
    sfg_wqp_property = folium.plugins.FeatureGroupSubGroup(fg_wqp_properties,name=property[:20],overlay=True,show=False);

    site_summary = summary_table.query("Property == '"+property+"'")
    
    for i, site in site_summary.iterrows():
        try:
            if (math.isnan(site.Avg)):
                continue
        except:
            continue
        coord = [site.Lat,site.Lon]
        value = float(site.Avg)
        #radius = 5
        radius = 7+2*abs(value-mean_val)/std_val
        color = val2color(min_val,value,max_val)
        fill_color = color
        label = "{0:s}: {1:s} = {2:.2f}".format(site.Site,property,value)
        html = label
        #html += '<br>min = {0:.2f}, max = {1:.2f}'.format(min_val,max_val)
        #html += '<br>color = '+color
        html += '<br>'+site.Site
        #html += '<br>'+row.Description
        html += '<br>Lat: {0:.2f}'.format(site.Lat)+', Lon: {0:.2f}'.format(site.Lon)
        iframe = folium.IFrame(html,width=width,height=height)
        popup = folium.Popup(iframe,max_width=max_width)
        _ = sfg_wqp_property.add_child(folium.CircleMarker(location=coord,radius=radius,color=color,opacity=0.7,fill=True,fill_color=fill_color,fill_opacity=0.7,popup=popup,tooltip=label)).add_to(river_map);
    sfg_wqp_property.add_child(fg_wqp_properties);
    
fg_wqp_properties.add_to(river_map);

### Add built-in basemaps

In [334]:
folium.TileLayer('StamenTerrain').add_to(river_map);
folium.TileLayer('OpenStreetMap').add_to(river_map);

### Add other basemap options using Map Servers at  ArcGIS Online

In [335]:
mapserver_dict = dict(
    NatGeo_World_Map=ARCGIS_WS+'/NatGeo_World_Map/MapServer',
    World_Street_Map=ARCGIS_WS+'/World_Street_Map/MapServer',
    World_Imagery=ARCGIS_WS+'/World_Imagery/MapServer',
    World_Topo_Map=ARCGIS_WS+'/World_Topo_Map/MapServer',
    World_Shaded_Relief=ARCGIS_WS+'/World_Shaded_Relief/MapServer',
    World_Terrain_Base=ARCGIS_WS+'/World_Terrain_Base/MapServer',
#   World_Physical_Map=ARCGIS_WS+'/World_Physical_Map/MapServer',  # only shows if zoomed out
)
mapserver_query = '/MapServer/tile/{z}/{y}/{x}'

for tile_name, tile_url in mapserver_dict.items():
    tile_url += mapserver_query
    _ = folium.TileLayer(tile_url,name=tile_name,attr=tile_name).add_to(river_map);

### Add the Layer Control widget

In [336]:
folium.LayerControl().add_to(river_map);

### Save the map as HTML

In [337]:
river_map.save(OUT_DIR+'/river_map.html')

<div id='map' />

## Display Map

In [338]:
from IPython.display import IFrame
IFrame(OUT_DIR+"/river_map.html", width=1500, height=750)

## USGS Gage Data

### Some defined constants
 * TODO: Move property limits to a dictionary

In [339]:
TURB_MIN = 0.0
TURB_MAX = 400.0
CHLOR_MIN = 0.0
CHLOR_MAX = 12.0
BGA_MIN = 0.05
BGA_MAX = 1.2
SPCOND_MIN = 100.0
SPCOND_MAX = 500.0
DO_MIN = 5.0
DO_MAX = 15.0
PAR_MIN = 0.0
PAR_MAX = 2000.0
NITR_MIN = 0.5
NITR_MAX = 1.5
GAGE_HGT0 = 31.0 # TODO: Need to get this from site elevation
FT_PER_M = 3.28

#### Get NWIS Sites (USGS Gage Stations) from database created for this river reach

<div id='gage_selection'></div>

### Get NWIS (USGS) Gage data directly from USGS (WaterData) web services 
* TODO:
  * Create dictionary of property codes, names, and order returned:
     *  168408 72254 Water velocity reading from field sensor, feet per second
     *  209867 72255 Mean water velocity for discharge computation, feet per second
     *  209869 00060 Discharge, cubic feet per second
     *  231500 00011 Temperature, water, degrees Fahrenheit, ADVM
     *   59675 00065 Gage height, feet
     *   59676 00010 Temperature, water, degrees Celsius
     *   59677 00400 pH, water, unfiltered, field, standard units
     *   59678 00095 Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius
     *   59679 00300 Dissolved oxygen, water, unfiltered, milligrams per liter
     *   59680 63680 Turbidity, water, unfiltered, monochrome near infra-red LED light, 780-900 nm, detection angle 90 +-2.5 degrees, formazin nephelometric units (FNU)
     *   59681 99133 Nitrate plus nitrite, water, in situ, milligrams per liter as nitrogen
* Varies with gage
* Order out does not match order of params in
* See: https://nwis.waterdata.usgs.gov/usa/nwis, https://waterservices.usgs.gov/rest/
* Can use dv instead of uv to get daily summary data (no timezone field)
* Can also receive data in WaterML and JSON formats
* Some USGS gages (e.g., USGS-03206000, Huntington) also collect precipitation data (cb_00045)

In [340]:
nwis_sites = spatialite_session.query(Site).filter_by(type='NWIS')
nwis_begin_date = "{0}-{1}-{2}".format(BEGIN_DATE.year,BEGIN_DATE.month,BEGIN_DATE.day)
nwis_end_date = "{0}-{1}-{2}".format(END_DATE.year,END_DATE.month,END_DATE.day)

In [341]:
site_names = []
for site in nwis_sites:
    site_names.append(site.name)
site_names

['USGS-03206000', 'USGS-03216000', 'USGS-03216070', 'USGS-03206000']

In [342]:
if (DB_NOT_FOUND):

    for nwis_site in nwis_sites:
        nwis_rt_stream = pd.read_csv("https://nwis.waterdata.usgs.gov/usa/nwis/uv/"+ \
                        "?site_no="+nwis_site.name[5:]+ \
                        "&period=&begin_date="+nwis_begin_date+"&end_date="+nwis_end_date+ \
                        "&cb_00065=on"+ \
                        "&format=rdb", \
                       sep='\t',comment='#',header=[0,1], \
                       dtype={1:object, 4:float}, \
                       na_values=['Eqp'])
        nwis_rt_stream = nwis_rt_stream.reset_index()
        nwis_rt_stream.columns = ['Index', 'Agency', 'Site', 'DateTime', 'TZ', 'Gage height', 'qa'] # TODO: Need to lookup property name from a dictionary
#        nwis_rt_stream = nwis_rt_stream.drop(nwis_rt_stream.columns[6:8:2],axis=1) # TODO: set upper limit based on number of properties requested/returned
#        nwis_rt_stream = nwis_rt_stream.drop(['Index','Agency','Site','TZ'],axis=1)
        
        for i, nwis_measurement in nwis_rt_stream.iterrows():
            try:
                value = float(nwis_measurement['Gage height'])
            except:
                continue
                
            measurement = Measurement(site_id=nwis_measurement['Site'],type='REALTIME',datetime=datetime.strptime(nwis_measurement['DateTime'],"%Y-%m-%d %H:%M"),property='Gage height',value=value)
            spatialite_session.add(measurement)

        spatialite_session.commit()

In [343]:
#nwis_measurements = pd.read_sql_query("select site_id as Site, ST_X(geom) as Lon, ST_Y(geom) as Lat, datetime as Datetime, property as Property, value as Value from measurement join site on site.name[5:] = site_id where measurement.type='REALTIME'", spatialite_engine)
nwis_measurements = pd.read_sql_query("select 'USGS-' || site_id as Site, datetime as Datetime, property as Property, value as Value from measurement where measurement.type='REALTIME'", spatialite_engine)
nwis_measurements = nwis_measurements[nwis_measurements['Value'].notna()]

In [344]:
#nwis_measurements

#### Index by Date-time

In [345]:
def datetime_nwis(row):
    pattern = '%Y-%m-%d %H:%M:%S.%f'
    dt = row['Datetime']
    return pd.to_datetime(time.mktime(time.strptime(dt,pattern)),unit='s')
    
nwis_measurements['DateTime'] = nwis_measurements.apply(lambda row: datetime_nwis(row),axis=1)
nwis_measurements = nwis_measurements.set_index(['DateTime'])

In [346]:
nwis_measurements['Year'] = nwis_measurements.index.year
nwis_measurements['Month'] = nwis_measurements.index.month

In [347]:
site_counts = nwis_measurements.groupby('Site').count()
property_counts = nwis_measurements.groupby('Property').count()

<div id='gage_plots'></div>

## USGS Gage Plots

### Distribution plots

In [348]:
def dist_plot_ts_gage(Site,Property,Kind='kde'):
    nwis_measurements.query("(Site == '"+Site+"') & (Property == '"+Property+"')")['Value'].plot(kind=Kind,color='blue')
        
interact(dist_plot_ts_gage, \
         Site=site_counts.index, \
         Property=property_counts.index, \
         Kind=['hist','kde','box'] \
        );

interactive(children=(Dropdown(description='Site', options=('USGS-03206000', 'USGS-03216000', 'USGS-03216070')…

#### Disable scrolling for time-serie plots

In [349]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

### Time-series plots
* TODO: also look at Bokeh for a stand-alone HTML plot

In [350]:
def plot_ts_gage(Property=property_counts.index[0]):
    nrows = len(site_counts)
    fig, axes = plt.subplots(nrows=nrows,ncols=1,figsize=(20,5*nrows))
    i = 0
    for site in site_counts.index:
        nwis_measurements.query("(Site == '"+site+"') & (Property == '"+Property+"')")['Value'].plot(ax=axes[i],drawstyle='steps-post',legend=True,color='blue')
        axes[i].legend([site],loc='upper left')
        i = i+1
            
interact(plot_ts_gage,
         Property=property_counts.index, \
        );

interactive(children=(Dropdown(description='Property', options=('Gage height',), value='Gage height'), Output(…

### Monthly box & whisker plots

In [351]:
# Using MatPlotLib
def gage_box_plots(Site,Property):
    fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(20,5),sharex=True)
    custom_means = dict(marker='D', markeredgecolor='black', markerfacecolor='firebrick')
    custom_box = dict(linestyle='-', linewidth=2, color='darkgoldenrod')
    nwis_measurements.query("(Site == '"+Site+"') & (Property == '"+Property+"')").boxplot(ax=ax,column=['Value'],by=['Year','Month'],grid=False,meanprops=custom_means,showmeans=True,meanline=False,boxprops=custom_box)
    plt.suptitle('')
    ax.set_title(Property+' @ '+Site, fontsize=16)
            
interact(gage_box_plots,
         Site=site_counts.index, \
         Property=property_counts.index \
        );

interactive(children=(Dropdown(description='Site', options=('USGS-03206000', 'USGS-03216000', 'USGS-03216070')…

In [352]:
# Using Seaborn
# Can't figure out how to use Month-Year as combo. Use hue nesting instead
def gage_box_plots2(Site,Property):
    fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(20,5),sharex=True)
    sbn.boxplot(ax = ax, data=nwis_measurements.query("(Site == '"+Site+"') & (Property == '"+Property+"')"), y='Value', x='Month', hue='Year')
    plt.suptitle('')
    ax.set_title(Property+' @ '+Site, fontsize=16)
                
interact(gage_box_plots2, \
         Site=site_counts.index, \
         Property=property_counts.index \
        );

interactive(children=(Dropdown(description='Site', options=('USGS-03206000', 'USGS-03216000', 'USGS-03216070')…